In [ ]:
# -- Switch to postgres user

# -- Create database

# -- Login as postgres user first
# -- psql -U postgres

# CREATE DATABASE bank_apps_reviews;

# CREATE USER bank_admin WITH PASSWORD 'admin123';

# GRANT ALL PRIVILEGES ON DATABASE bank_apps_reviews TO bank_admin;



# -- Login using the new user:
# psql -U bank_admin -d bank_apps_reviews -h localhost



# -- Create the tables


# -- Banks table

#CREATE TABLE banks (
#    bank_id SERIAL PRIMARY KEY,
#    app_id VARCHAR(255) UNIQUE NOT NULL,
#    title VARCHAR(255),
#    score DECIMAL(3,2),
#    ratings INT,
#    reviews INT,
#    installs VARCHAR(50),
#    bank_code VARCHAR(20) UNIQUE,
#    bank_name VARCHAR(255)
#);



# -- Reviews table

#CREATE TABLE reviews (
#    review_id UUID PRIMARY KEY,
#    review_text TEXT,
#    rating INT CHECK (rating BETWEEN 1 AND 5),
#    review_date DATE,
#    review_year INT,
#    review_month INT,
#    bank_code VARCHAR(20),
#    bank_name VARCHAR(255),
#    user_name VARCHAR(255),
#    thumbs_up INT,
#    text_length INT,
#    source VARCHAR(100),
#
#    CONSTRAINT fk_bank
#      FOREIGN KEY (bank_code)
#      REFERENCES banks (bank_code)
#      ON DELETE CASCADE
#);




# --check the tables

# \d


# -- List all tables

# \dt

# -- Describe a table (check schema)

# \d table_name


# -- Delete all records

# TRUNCATE TABLE table_name RESTART IDENTITY CASCADE;


In [26]:
# pip install psycopg2-binary
import pandas as pd
import psycopg2

In [27]:
# Read CSV files into pandas DataFrames
df_banks = pd.read_csv("../data/processed/app_info.csv")
df_reviews = pd.read_csv("../data/processed/reviews_processed.csv")

In [32]:
# -------------------------------------------------------
# Connect to PostgreSQL database
# -------------------------------------------------------

# Establish a connection to PostgreSQL using psycopg2
# You must make sure the database 'restaurant_reviews' is already created
conn = psycopg2.connect(
    host="localhost",              # PostgreSQL is running on your machine
    database="bank_apps_reviews", # The database name you created
    user="bank_admin",                  # Your PostgreSQL username
    password="admin123"            # Your PostgreSQL password
)

# Create a cursor object — used to execute SQL commands
cur = conn.cursor()

In [33]:
# -------------------------------------------------------
# Insert data into banks table
# -------------------------------------------------------

# Loop through each row in df_banks
for _, row in df_banks.iterrows():
    cur.execute("""
        INSERT INTO banks (app_id, title, score, ratings, reviews, installs, bank_code, bank_name)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        str(row["app_id"]),
        str(row["title"]),
        float(row["score"]),
        int(row["ratings"]),
        int(row["reviews"]),
        str(row["installs"]),
        str(row["bank_code"]),
        str(row["bank_name"])
    ))
conn.commit()
print("Banks inserted!")

Banks inserted!


In [34]:

# -------------------------------------------------------
# Insert data into reviews table
# -------------------------------------------------------

# Loop through each row in df_reviews
for _, row in df_reviews.iterrows():
    cur.execute("""
        INSERT INTO reviews 
        (review_id, review_text, rating, review_date, review_year, review_month,
         bank_code, bank_name, user_name, thumbs_up, text_length, source)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        str(row["review_id"]),
        str(row["review_text"]),
        int(row["rating"]),
        row["review_date"],
        int(row["review_year"]),
        int(row["review_month"]),
        str(row["bank_code"]),
        str(row["bank_name"]),
        str(row.get("user_name", "anonymous")),
        int(row.get("thumbs_up", 0)),
        int(row.get("text_length", len(str(row["review_text"])))),
        str(row["source"])
    ))
conn.commit()
print("Reviews inserted!")

cur.close()
conn.close()
print("Done!")


Reviews inserted!
Done!
